# GT Educação

O GT Educação enviou uma série de formulários com indicadores desejados para o painel. A maioria desses indicadores estão disponíveis no Portal de Dados Abertos da Prefeitura de São Paulo (http://dados.prefeitura.sp.gov.br/).

Portanto, vamos carregar o módulo de downloads do portal de dados abertos.

In [ ]:
import os
import pandas as pd

from core.downloads import dadosabertos as da
from core.downloads import geosampa as gs

# Extração e transformação inicial

## Número de alunos da Rede Municipal de Ensino

O primeiro indicador é o número de alunos da rede municipal, segmentado por nível educacional (Creche, Pré Escola, Ensino Fundamental I, Ensino Fundamental II, EJA I, EJA II, Ensino Médio, Ed. Prof.).

O conjunto de dados que contém esse indicador está disponível em http://dados.prefeitura.sp.gov.br/dataset/demanda-e-matriculas. Sabendo que o último trecho da url representa a id do conjunto de dados, vamos utilizá-la para fazer a extração dos dados do portal.

Primeiro, precisamos visualizar todos os recursos relativos a esse conjunto de dados. Para padronizar o período de todos os conjuntos, vamos focar em dezembro de 2023.

In [ ]:
pkg_name = 'demanda-e-matriculas'
resources = da.package_resources(pkg_name, '23')
resources

Sabendo o id do arquivo de dezembro de 2023, vamos carregar o recurso como um dataframe.

In [ ]:
resource_id = 'a63afccb-1a19-4438-96b1-7a13c4ccb586'
mat_23 = da.load_resource(resource_id, pandas_kwargs=dict(header=[0,1]))
mat_23

O dataframe contem dados de matrículas, matrículas em processo e demanda não atendida. Como desejamos apenas matrículas, vamos remover as outras colunas.

In [ ]:
mat_23 = mat_23[['Distrito', 'Matrículas']]
mat_23

Agora, sabendo que temos apenas dados sobre matrículas, vamos ajustar as colunas.

In [ ]:
mat_23 = (mat_23
 .droplevel(0, axis='columns')
 .rename(columns={'Unnamed: 0_level_1': 'Distrito'}))

mat_23

As últimas linhas do dataframe são apenas dados de totalização e notas, então vamos excluí-las também.

In [ ]:
mat_23 = mat_23.loc[mat_23['Distrito'].str.lower()!='total']
mat_23 = mat_23.loc[~mat_23.iloc[:,1].isna()]
mat_23

Finalmente, vamos "despivotar" a tabela, transformando o nome das colunas em uma nova coluna chamada "Nível educacional" e unificando as colunas de valor sob uma coluna chamada matrículas.

In [ ]:
mat_23 = mat_23.melt(
    id_vars='Distrito',
    var_name='Nível Educacional',
    value_name='Matrículas'
)
mat_23

## Número de professores da Rede Municipal de Ensino

O segundo indicador solicitado é o número de professores da Rede Municipal de Ensino, com recortes por Raça, Sexo, Nível de Ensino (Educação Infantil, Ensino Fundamental etc.), Tipo de Unidade de atuação (EMEF, EMEI etc.).

O conjunto de dados que contém esse indicador está disponível em http://dados.prefeitura.sp.gov.br/dataset/microdados-servidores-perfil, porém os arquivos possuem informação sobre todos os servidores da SME, o que inclui os servidores administrativos, que precisarão ser excluídos.

Primeiro, precisamos visualizar todos os recursos relativos a esse conjunto de dados. Para padronizar o período de todos os conjuntos, vamos focar em dezembro de 2023.

In [ ]:
pkg_name = 'microdados-servidores-perfil'
resources = da.package_resources(pkg_name, '23')
resources

Aparentemente, os dados de perfil dos servidores possuem periodicidade anual, já que temos apenas um arquivo disponível para 2023.

In [ ]:
resource_id = resources[0]['id']
perf_23 = da.load_resource(resource_id,
                           pandas_kwargs={'encoding': 'utf-8'})
perf_23

Com os dados baixados, vamos selecionar as colunas para atender às segmentações propostas no formulário.

In [ ]:
# Raça, Sexo, Nível de Ensino (Educação Infantil, Ensino Fundamental etc.), Tipo de Unidade de atuação (EMEF, EMEI etc.).

perf_cols = [
    'CD_SEXO',
    'CD_RACA_COR',
    'DC_RACA_COR',
    'CD_DEF',
    'CD_CARGO_BASE',
    'DC_CARGO_BASE',
    'DT_INICIO_CARGO_BASE',
    'CD_CARGO_ATUAL',
    'DC_CARGO_ATUAL',
    'CD_AREA_ATUACAO_ATUAL',
    'DC_AREA_ATUACAO_ATUAL',
    'CD_UNIDADE_ATUAL',
    'TP_UNIDADE_ATUAL',
    'DC_UNIDADE_ATUAL',
]

perf_23 = perf_23.loc[:, perf_cols]
perf_23

O formulário solicita também a exclusão de professores aposentados e servidores administrativos do conjunto de dados. Apesar de existir uma coluna com um valor "aposentado", a `ACUMULO_SITUACAO_SEGUNDO_VINCULO`, essa coluna parece se referir à situação do segundo vínculo, e não do vínculo com a PMSP. Além disso, a lista de servidores aposentados da Secretaria Municipal de Educação tem mais de 94 mil registros, então esses registros provavelmente não estão nos dados de perfil dos servidores.

Para excluir os servidores administrativos, vamos primeiro avaliar quais cargos estão presentes na base.

In [ ]:
df_cargos = perf_23[['CD_CARGO_BASE', 'DC_CARGO_BASE']].copy()
df_cargos = df_cargos.drop_duplicates().reset_index(drop=True)

df_cargos

Dado a quantidade de cargos, vamos fitrar a lista com base em uma regra geral, mantendo apenas os cargos que possual a string "PROF" em sua descrição.

In [ ]:
filtro_prof = df_cargos['DC_CARGO_BASE'].str.lower().str.contains('prof.', regex=False)
df_cargos_prof = df_cargos.loc[filtro_prof]

df_cargos_prof

Por último, filtramos a base geral com base nessa lista de cargos.

In [ ]:
perf_23 = (
    perf_23
    .loc[
        perf_23['DC_CARGO_BASE'].isin(df_cargos_prof['DC_CARGO_BASE'].tolist())
    ]
)

perf_23

## Número de Unidades Escolares

O terceiro indicador solicitado é o número de Unidades Escolares da Rede Municipal de Ensino, com recortes por Tipo de Escola (EMEI, EMEF etc.) e Distrito.

O conjunto de dados que contém esse indicador está disponível em http://dados.prefeitura.sp.gov.br/dataset/cadastro-de-escolas-municipais-conveniadas-e-privadas.

Primeiro, precisamos visualizar todos os recursos relativos a esse conjunto de dados. Para padronizar o período de todos os conjuntos, vamos focar em dezembro de 2023.

In [ ]:
pkg_name = 'cadastro-de-escolas-municipais-conveniadas-e-privadas'
resources = da.package_resources(pkg_name, '23')
resources

Aparentemente, os dados de unidades escolares possuem periodicidade anual, já que temos apenas um arquivo disponível para 2023.

In [ ]:
resource_id = resources[0]['id']
escolas_23 = da.load_resource(resource_id)
escolas_23

Os dados das escolas também serão usados com outros conjuntos de dados para regionalizá-los, então vamos manter também algumas colunas de identificação das escolas para facilitar esse uso o posterior. Vamos também filtrar apenas as escolas ativas.

In [ ]:
escolas_cols = ['CODESC',
                'TIPOESC',
                'NOMES',
                'SUBPREF',
                'DISTRITO',
                'CODINEP',
                'CD_CIE',
                'NOME_ANT',
                'REDE']

escolas_filtro = escolas_23['SITUACAO'].str.lower().str.contains('ativa')
escolas_23 = escolas_23.loc[escolas_filtro, escolas_cols]

escolas_23

## Perfil dos alunos

O quinto indicador solicitado é o número de alunos da Rede Municipal de Ensino, com recortes por Raça, Sexo, existência de Necessidades Especiais, Distrito, Rede de Ensino (Administração Direta ou Rede Conveniada/Parceira).

O conjunto de dados que contém esse indicador está disponível em http://dados.prefeitura.sp.gov.br/dataset/perfil-dos-educandos-cor-raca-idade-sexo-necessidades-educacionais-especiais.

Primeiro, precisamos visualizar todos os recursos relativos a esse conjunto de dados. Para padronizar o período de todos os conjuntos, vamos focar em dezembro de 2023.

In [ ]:
pkg_name = 'perfil-dos-educandos-cor-raca-idade-sexo-necessidades-educacionais-especiais'
resources = da.package_resources(pkg_name, '23')
resources

Aparentemente, os dados de perfil dos educandos possuem periodicidade anual, já que temos apenas um arquivo disponível para 2023.

In [ ]:
resource_id = resources[0]['id']
alunos_23 = da.load_resource(resource_id)
alunos_23

In [ ]:
alunos_cols = ['DISTRITO',
               'REDE',
               'MODAL',
               'SEXO',
               'NEE',
               'RACA',
               'Qtde']

alunos_23 = alunos_23[alunos_cols]

alunos_23

## Número de alunos estrangeiros

O sétimo indicador é o número de alunos estrangeiros da rede municipal, segmentado por País de Procedência, Distrito, Tipo de Rede (Direta ou Conveniada/Parceira), Etapa (Creche, fundamental, EJA etc.).

O conjunto de dados que contém esse indicador está disponível em http://dados.prefeitura.sp.gov.br/dataset/educandos-estrangeiros-por-nacionalidade. Sabendo que o último trecho da url representa a id do conjunto de dados, vamos utilizá-la para fazer a extração dos dados do portal.

Primeiro, precisamos visualizar todos os recursos relativos a esse conjunto de dados.

In [ ]:
pkg_name = 'educandos-estrangeiros-por-nacionalidade'
resources = da.package_resources(pkg_name, '23')
resources

Os dados de alunos estrangeiros parecem estar disponíveis anualmente, mas no mês de junho. Como não existe arquivo para dezembro de 2023, vamos utilizar o arquivo de junho de 2024.

In [ ]:
pkg_name = 'educandos-estrangeiros-por-nacionalidade'
resources = da.package_resources(pkg_name, '24')
resources

In [ ]:
resource_id = resources[2]['id']
alunos_est_2024 = da.load_resource(resource_id)
alunos_est_2024

In [ ]:
# Recortes: País de Procedência, Distrito, Tipo de Rede (Direta ou Conveniada/Parceira), Etapa (Creche, fundamental, EJA etc.)
alunos_bf_cols = ['DISTRITO',
                   'COD_PAIS',
                   'NOME_PAIS',
                   'ETAPA',
                   'REDE',
                   'QTDE']

alunos_est_2024 = alunos_est_2024[alunos_bf_cols]

alunos_est_2024

## Número de alunos beneficiários do Bolsa Família

O oitavo indicador é o número de alunos beneficiários do Bolsa Família da rede municipal, segmentado por Distrito e nível de ensino.

O conjunto de dados que contém esse indicador está disponível em http://dados.prefeitura.sp.gov.br/dataset/beneficiarios-programa-bolsa-familia-creches-municipais-e-conveniadas. Sabendo que o último trecho da url representa a id do conjunto de dados, vamos utilizá-la para fazer a extração dos dados do portal.

As notas (que podem ser vistas na saída abaixo) desse conjunto de dados incluem a informação de que a estrutura do conjunto mudou a partir de 2024 e, por isso, foi criado um novo conjunto de dados com a estrutura mais recente, disponível em http://dados.prefeitura.sp.gov.br/dataset/beneficiarios-do-programa-bolsa-familia-por-etapa-de-ensino-da-rede-municipal-de-educacao. Vamos utilizar essa versão mais recente e os dados de julho de 2024, assim como o número de alunos estrangeiros.

In [ ]:
pkg_name = 'beneficiarios-programa-bolsa-familia-creches-municipais-e-conveniadas'
pkg = da.package_show(pkg_name)
print(pkg['notes'])

In [ ]:
pkg_name = 'beneficiarios-do-programa-bolsa-familia-por-etapa-de-ensino-da-rede-municipal-de-educacao'
resources = da.package_resources(pkg_name, '24')
resources

In [ ]:
resource_id = resources[2]['id']
alunos_bf_2024 = da.load_resource(resource_id)
alunos_bf_2024

In [ ]:
# Recortes: Alunos Beneficiários do Bolsa-Família por distrito, por nível de Ensino
alunos_bf_cols = ['CODEOL',
                   'TIPO',
                   'UNIDADE',
                   'DIRETORIA',
                   'DISTRITO',
                   'CRECHE',
                   'PRE ESCOLA',
                   'ENSINO FUNDAMENTAL',
                   'EDUCAÇÃO PROFISSIONAL',
                   'ENSINO MEDIO',
                   'EDUCAÇÃO ESPECIAL']

alunos_bf_2024 = alunos_bf_2024[alunos_bf_cols]

alunos_bf_2024

In [ ]:
alunos_bf_id_vars = alunos_bf_cols[0:5]

alunos_bf_2024 = alunos_bf_2024.melt(id_vars=alunos_bf_id_vars,
                    var_name='NIVEL',
                    value_name='QTDE')

alunos_bf_2024

## Distritos e Subprefeituras

Para agregarmos os dados em Subprefeituras, precisamos de uma lista de distritos e subprefeituras. A lista de distritos e subprefeituras pode ser obtida no GeoSampa, que é o portal de dados geográficos da Prefeitura de São Paulo.

In [ ]:
gs.get_capabilities('distrito')

In [ ]:
df_dist = gs.get_features('geoportal:distrito_municipal')
df_dist

In [ ]:
gs.get_capabilities('subprefeitura')

In [ ]:
df_subs = gs.get_features('geoportal:subprefeitura')
df_subs

# Transformação e mesclagem de dados

Para facilitar o trabalho dos dados no Qlik Sense, vamos avaliar como é a melhor forma de retrabalhar os dados de acordo com o paradigma fato-dimensão.

## Nível Escolar

A primeira dimensão que parece fazer sentido é o Nível escolar. Vamos avaliar como aparece em cada tabela.

In [ ]:
mat_23['Nível Educacional'].value_counts()

In [ ]:
perf_23[['CD_AREA_ATUACAO_ATUAL', 'DC_AREA_ATUACAO_ATUAL']].value_counts()

In [ ]:
alunos_23['MODAL'].value_counts()

In [ ]:
alunos_est_2024['ETAPA'].value_counts()

In [ ]:
alunos_bf_2024['NIVEL'].value_counts()

Existe uma tabela de correspondência relacionada a essas categorias fornecida no último relatório de Gestão da Função Educação. A tabela é a seguinte:

| Nível Educacional | Tipo de escola |
| --- | --- |
| Educação Infantil | Creches |
| Educação Infantil | Pré-escola |
| Ensino Fundamental | EMEF |
| Ensino Médio | EMEFM |
| Educação de Jovens e Adultos | EJA/CIEJA |
| Educação de Jovens e Adultos | MOVA (Parceria) |
| Educação de Jovens e Adultos | Educação Profissional |
| Educação Especial | EMEBS |
| Educação Especial | Educação Especial (Parceira) |


Essa tabela permite a padronização de quase todas as tabelas, exceto a de perfil dos professores. Para as outras, vamos utilizar a tabela como base para a criação de um dicionário que será utilizado como *mapper* em cada uma das tabelas.

In [ ]:
mat_23_nivel = {
    'Creche': 'Educação Infantil',
    'Pré Escola': 'Educação Infantil',
    'Ens. Fund.I': 'Ensino Fundamental',
    'Ens. Fund.II': 'Ensino Fundamental',
    'EJA I': 'Educação de Jovens e Adultos',
    'EJA II': 'Educação de Jovens e Adultos',
    'Ens. Médio': 'Ensino Médio',
    'Ed. Prof.': 'Educação de Jovens e Adultos'
}

mat_23['Nível padronizado'] = mat_23['Nível Educacional'].map(mat_23_nivel)
mat_23['Nível padronizado'].value_counts()

In [ ]:
alunos_23_nivel = {
    'CRECHE': 'Educação Infantil',
    'PRE': 'Educação Infantil',
    'FUND': 'Ensino Fundamental',
    'EJA': 'Educação de Jovens e Adultos',
    'MOVA': 'Educação de Jovens e Adultos',
    'MEDIO': 'Ensino Médio',
    'EDPROF': 'Educação de Jovens e Adultos',
    'ESPEC': 'Educação Especial',
    'CONVEE': 'Educação Especial',
    'ATCOMP': '?',
    'REC': '?',
    'SAAI': '?'
}

alunos_23['Nível padronizado'] = alunos_23['MODAL'].map(alunos_23_nivel)
alunos_23['Nível padronizado'].value_counts()

In [ ]:
alunos_est_2024_nivel = {
    'CRECHE': 'Educação Infantil',
    'PRE': 'Educação Infantil',
    'FUND': 'Ensino Fundamental',
    'EJA': 'Educação de Jovens e Adultos',
    'MOVA': 'Educação de Jovens e Adultos',
    'MEDIO': 'Ensino Médio',
    'EDPROF': 'Educação de Jovens e Adultos',
    'ESPEC': 'Educação Especial',
    'CONVEE': 'Educação Especial'
}

alunos_est_2024['Nível padronizado'] = alunos_est_2024['ETAPA'].map(alunos_est_2024_nivel)
alunos_est_2024['Nível padronizado'].value_counts()

In [ ]:
alunos_bf_2024_nivel = {
    'CRECHE': 'Educação Infantil',
    'PRE ESCOLA': 'Educação Infantil',
    'ENSINO FUNDAMENTAL': 'Ensino Fundamental',
    'ENSINO MEDIO': 'Ensino Médio',
    'EDUCAÇÃO PROFISSIONAL': 'Educação de Jovens e Adultos',
    'EDUCAÇÃO ESPECIAL': 'Educação Especial'
}

alunos_bf_2024['Nível padronizado'] = alunos_bf_2024['NIVEL'].map(alunos_bf_2024_nivel)
alunos_bf_2024['Nível padronizado'].value_counts()

## Padronização dos nomes de Distritos

Os dados de matrículas, escolas, educandos, educandos estrangeiros e educandos beneficiários do Bolsa Família possuem uma coluna chamada "Distrito" que contém o nome do distrito onde a escola está localizada.

Já os dados de servidores não possuem dados de distrito, mas possuem o nome da escola, que pode ser utilizada para buscar o distrito correspondente. Antes de unir as tabelas para fazer isso, vamos padronizar os nomes dos distritos de acordo com os dados do geosampa.

### Número de Unidades Escolares

In [ ]:
(
    escolas_23
    .loc[~escolas_23['DISTRITO'].isin(df_dist['nm_distrito_municipal'])]
    ['DISTRITO']
    .unique().tolist()
)

### Perfil dos alunos

In [ ]:
(
    alunos_23
    .loc[~alunos_23['DISTRITO'].isin(df_dist['nm_distrito_municipal'])]
    ['DISTRITO']
    .unique().tolist()
)

### Número de alunos estrangeiros

In [ ]:
(
    alunos_est_2024
    .loc[~alunos_est_2024['DISTRITO'].isin(df_dist['nm_distrito_municipal'])]
    ['DISTRITO']
    .unique().tolist()
)

### Número de alunos beneficiários do Bolsa Família

In [ ]:
(
    alunos_bf_2024
    .loc[~alunos_bf_2024['DISTRITO'].isin(df_dist['nm_distrito_municipal'])]
    ['DISTRITO']
    .unique().tolist()
)

Nenhum dos conjuntos de dados possui distritos que não estejam na lista do GeoSampa.

Agora, vamos unir os dados de servidores com os dados de escolas para adicionar a coluna de distrito aos dados de servidores.

### Número de professores da Rede Municipal de Ensino

Primeiro, vamos criar uma coluna com o tipo de escola e o nome da escola nos dois dataframes, para que possamos unir os dados de servidores com os dados de escolas.

In [ ]:
escolas_23['NOMES_COMPLETO'] = (
    escolas_23['TIPOESC'].str.strip().str.upper()
    + ' ' + escolas_23['NOMES'].str.strip().str.upper()
)
escolas_23['NOMES_COMPLETO']

In [ ]:
perf_23['DC_UNIDADE_ATUAL_COMPLETA'] = (
    perf_23['TP_UNIDADE_ATUAL'].str.strip().str.upper()
    + ' ' + perf_23['DC_UNIDADE_ATUAL'].str.strip().str.upper()
)
perf_23['DC_UNIDADE_ATUAL_COMPLETA']

In [ ]:
perf_23 = (
    perf_23
    .merge(escolas_23[['NOMES_COMPLETO', 'DISTRITO']],
           left_on=['DC_UNIDADE_ATUAL_COMPLETA'],
           right_on=['NOMES_COMPLETO'],
           how='left')
)

perf_23

Vamos avaliar se todas as unidades escolares foram encontradas no dataframe de unidades.

In [ ]:
(
    perf_23
    .query('DISTRITO.isnull()')
)

In [ ]:
(
    perf_23
    .query('DISTRITO.isna()')
    .shape[0]
    /perf_23.shape[0]
)

Cerca de 3,6% dos professores não tiveram as unidades escolares encontradas. Vamos avaliar quais são essas unidades.

In [ ]:
(
    perf_23
    .query('DISTRITO.isna()')
    [['TP_UNIDADE_ATUAL', 'DC_UNIDADE_ATUAL']]
    .sort_values(['TP_UNIDADE_ATUAL', 'DC_UNIDADE_ATUAL'])
    .drop_duplicates()
)

Por ora, vamos manter dessa forma e, caso seja necessário, podemos avaliar como tratar esses casos numa atualização futura.

## Adicionando as Subprefeituras

Agora, vamos adicionar a coluna de subprefeitura a todos os dataframes.

### Associando distritos e subprefeituras

In [ ]:
df_dist.sort_values('cd_identificador_subprefeitura').head(2)

In [ ]:
df_subs.sort_values('cd_identificador_subprefeitura').head(2)

In [ ]:
df_subs.sort_values('cd_subprefeitura').head(2)

Os dados de distritos e subprefeituras não estão completamente compatíveis, então precisamos de um tratamento adicional.

O dataframe de distritos possui uma coluna chamada `cd_identificador_subprefeitura`, mas não corresponde exatamente à coluna `cd_identificador_subprefeitura` do dataframe de subprefeituras. Ele corresponde parcialmente à coluna `cd_subprefeitura` do dataframe de subprefeituras, mas não é exatamente a mesma, porque esta última possui um zero à esquerda que não está presente no dataframe de distritos. Portanto, vamos criar uma coluna com o código de subprefeitura no dataframe de distritos, que será o código de subprefeitura com o zero à esquerda.

In [ ]:
df_dist['cd_subprefeitura'] = (
    df_dist['cd_identificador_subprefeitura'].astype(str)
    .str.zfill(2)
)

df_dist.sort_values('cd_subprefeitura').head(2)

Agora, vamos associar os distritos e subprefeituras.

In [ ]:
df_dist_sub = df_dist.merge(
    df_subs[['cd_subprefeitura', 'nm_subprefeitura']],
    on='cd_subprefeitura',
    how='left'
)
df_dist_sub.sort_values('cd_subprefeitura')

Aparentemente, os dados de distritos e subprefeituras estão compatíveis, então vamos adicionar a coluna de subprefeitura aos outros dataframes.

### Número de Unidades Escolares

In [ ]:
escolas_23 = escolas_23.merge(
    df_dist_sub[['nm_distrito_municipal', 'cd_subprefeitura', 'nm_subprefeitura']],
    left_on='DISTRITO',
    right_on='nm_distrito_municipal',
    how='left'
)

escolas_23

In [ ]:
escolas_23[escolas_23['nm_subprefeitura'].isna()]

### Perfil dos alunos

In [ ]:
alunos_23 = alunos_23.merge(
    df_dist_sub[['nm_distrito_municipal', 'cd_subprefeitura', 'nm_subprefeitura']],
    left_on='DISTRITO',
    right_on='nm_distrito_municipal',
    how='left'
)

alunos_23

In [ ]:
alunos_23[alunos_23['nm_subprefeitura'].isna()]

### Número de alunos estrangeiros

In [ ]:
alunos_est_2024 = alunos_est_2024.merge(
    df_dist_sub[['nm_distrito_municipal', 'cd_subprefeitura', 'nm_subprefeitura']],
    left_on='DISTRITO',
    right_on='nm_distrito_municipal',
    how='left'
)

alunos_est_2024

In [ ]:
alunos_est_2024[alunos_est_2024['nm_subprefeitura'].isna()]

### Número de alunos beneficiários do Bolsa Família

In [ ]:
alunos_bf_2024 = alunos_bf_2024.merge(
    df_dist_sub[['nm_distrito_municipal', 'cd_subprefeitura', 'nm_subprefeitura']],
    left_on='DISTRITO',
    right_on='nm_distrito_municipal',
    how='left'
)

alunos_bf_2024

In [ ]:
alunos_bf_2024[alunos_bf_2024['nm_subprefeitura'].isna()]

### Número de professores da Rede Municipal de Ensino

In [ ]:
perf_23 = perf_23.merge(
    df_dist_sub[['nm_distrito_municipal', 'cd_subprefeitura', 'nm_subprefeitura']],
    left_on='DISTRITO',
    right_on='nm_distrito_municipal',
    how='left'
)

perf_23

In [ ]:
perf_23[perf_23['nm_subprefeitura'].isna() & ~perf_23['DISTRITO'].isna()]

## Adicionando tempo de atuação dos educadores na SME

In [ ]:
ref_date = pd.to_datetime('2023-12-31')
ref_date

In [ ]:
perf_23['DT_INICIO_CARGO_BASE'] = pd.to_datetime(perf_23['DT_INICIO_CARGO_BASE'], format='%d/%m/%Y')
perf_23

In [ ]:
perf_23['TEMPO_ATUACAO_SME'] =  ref_date - perf_23['DT_INICIO_CARGO_BASE']

perf_23

In [ ]:
perf_23['DIAS_ATUACAO_SME'] =  perf_23['TEMPO_ATUACAO_SME'].dt.days

perf_23

In [ ]:
perf_23['ANOS_ATUACAO_SME'] =  perf_23['TEMPO_ATUACAO_SME'].dt.days / 365.25
perf_23['ANOS_ATUACAO_SME'] = round(perf_23['ANOS_ATUACAO_SME'], 2)

perf_23

In [ ]:
perf_23 = perf_23.drop(columns='TEMPO_ATUACAO_SME')
perf_23

# Armazenamento dos dados

Finalmente, salvamos os arquivos como csv para utilizarmos no Qlik Sense.

In [ ]:
base_path = os.path.join('data_output', 'educacao')

if not os.path.exists(base_path):
    os.makedirs(base_path)

for name, df in [('demanda e matriculas', mat_23),
                 ('servidores-perfil', perf_23),
                 ('escolas-municipais', escolas_23),
                 ('perfil-dos-educandos', alunos_23),
                 ('educandos-estrangeiros', alunos_est_2024),
                 ('beneficiarios-pbf', alunos_bf_2024)]:
    
    filepath = os.path.join(base_path, f'{name}.csv')

    df.to_csv(filepath,
              index=False,
              sep=';',
              decimal=',',
              encoding='utf-8')